In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from nltk.corpus import stopwords
import string
def process_text(s):

    # Check string to see if they are a punctuation
    nopunc = [char for char in s if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_string


In [17]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test_1.csv')

In [18]:
# Tokenize the text :Convert the normal text strings in to a list of tokens (words that we actually want)
#rerun, takes LOOOONG
train['Clean Text'] = train['Statement'].apply(process_text)
test['Clean Text'] = test['Statement'].apply(process_text)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=process_text).fit(train['Clean Text'])

print(len(bow_transformer.vocabulary_)) #Total vocab words


#Bag-of-Words (bow) transform the entire DataFrame of text
train_bow = bow_transformer.transform(train['Clean Text'])
print('Shape of Sparse Matrix: ', train_bow.shape)
print('Amount of Non-Zero occurences: ', train_bow.nnz)

test_bow = bow_transformer.transform(test['Clean Text'])
print('Shape of Sparse Matrix: ', test_bow.shape)
print('Amount of Non-Zero occurences: ', test_bow.nnz)




10218
Shape of Sparse Matrix:  (10240, 10218)
Amount of Non-Zero occurences:  10240
Shape of Sparse Matrix:  (2551, 10218)
Amount of Non-Zero occurences:  14


In [30]:
sparsity = (100.0 * train_bow.nnz / (train_bow.shape[0] * train_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity2 = (100.0 * test_bow.nnz / (test_bow.shape[0] * test_bow.shape[1]))
print('sparsity2: {}'.format(round(sparsity2)))

sparsity: 0
sparsity2: 0


In [31]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(train_bow)
news_tfidf = tfidf_transformer.transform(train_bow)
print(news_tfidf.shape)

testnews_tfidf = tfidf_transformer.transform(test_bow)
print(testnews_tfidf.shape)

(10240, 10218)
(2551, 10218)


In [32]:

from sklearn.naive_bayes import MultinomialNB
fakenews_detect_model = MultinomialNB().fit(news_tfidf, train['Label'])

In [33]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

[False  True  True ...  True False False]


In [34]:
from sklearn.metrics import classification_report
print (classification_report(train['Label'], predictions))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      4488
        True       1.00      1.00      1.00      5752

    accuracy                           1.00     10240
   macro avg       1.00      1.00      1.00     10240
weighted avg       1.00      1.00      1.00     10240



In [35]:
predictions = fakenews_detect_model.predict(testnews_tfidf)
print(predictions)

[ True  True  True ...  True  True  True]


In [37]:
from sklearn.metrics import classification_report
print (classification_report(test['Label'], predictions))

              precision    recall  f1-score   support

       False       0.90      0.01      0.02      1169
        True       0.54      1.00      0.70      1382

    accuracy                           0.54      2551
   macro avg       0.72      0.50      0.36      2551
weighted avg       0.71      0.54      0.39      2551



In [38]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
clf = svm.SVC(kernel='linear')
clf.fit(news_tfidf,train['Label'])

SVC(kernel='linear')

In [41]:
y_pred_test = clf.predict(testnews_tfidf)

print('Kernel: linear')


print()
print('Testing')
print(classification_report(test['Label'], y_pred_test))


Kernel: linear

Testing
              precision    recall  f1-score   support

       False       0.90      0.01      0.02      1169
        True       0.54      1.00      0.70      1382

    accuracy                           0.54      2551
   macro avg       0.72      0.50      0.36      2551
weighted avg       0.71      0.54      0.39      2551



In [85]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(train,train, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(6860, 3) (3380, 3) (6860, 3) (3380, 3)


In [87]:
# test part of training 
X_test['data'] = X_test['Statement'].apply(process_text)
X_test_bow = bow_transformer.transform(X_test['data'])
print('Shape of Sparse Matrix: ', X_test_bow.shape)
print('Amount of Non-Zero occurences: ', X_test_bow.nnz)

X_testnews_tfidf = tfidf_transformer.transform(X_test_bow)
print(X_testnews_tfidf.shape)

y_pred_test = clf.predict(X_testnews_tfidf)

print('Kernel: linear')


print()
print('Testing')
print(classification_report(y_test['Label'], y_pred_test))

<ipython-input-87-1d977dc95174>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['data'] = X_test['Statement'].apply(process_text)


Shape of Sparse Matrix:  (3380, 10218)
Amount of Non-Zero occurences:  3380
(3380, 10218)
Kernel: linear

Testing
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1508
        True       1.00      1.00      1.00      1872

    accuracy                           1.00      3380
   macro avg       1.00      1.00      1.00      3380
weighted avg       1.00      1.00      1.00      3380



In [63]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
clf = knn.fit(news_tfidf,train['Label'])
from sklearn.metrics import classification_report
predicted = clf.predict(testnews_tfidf)
print (classification_report(test['Label'], predicted))

              precision    recall  f1-score   support

       False       0.46      1.00      0.63      1169
        True       1.00      0.00      0.01      1382

    accuracy                           0.46      2551
   macro avg       0.73      0.50      0.32      2551
weighted avg       0.75      0.46      0.29      2551



In [60]:

knn = KNeighborsClassifier(n_neighbors=10)
clf = knn.fit(news_tfidf,train['Label'])
from sklearn.metrics import classification_report
predicted = clf.predict(testnews_tfidf)
print (classification_report(test['Label'], predicted))

              precision    recall  f1-score   support

       False       1.00      0.00      0.00      1169
        True       0.54      1.00      0.70      1382

    accuracy                           0.54      2551
   macro avg       0.77      0.50      0.35      2551
weighted avg       0.75      0.54      0.38      2551

